In [18]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns


## CLEANING DATASET

In [19]:
df = pd.read_csv('Alonhadat.csv',encoding='utf-8')
df

,price,area,road_width,floors,bedroom,loc,garage
0,": 3,8 tỷ",50 m\n2,7m,2 lầu,2 phòng ngủ,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
1,": 9,7 tỷ",100 m\n2,6m,2 lầu,3 phòng ngủ,"Đường Trần Tống, Phường Thạc Gián, Quận Thanh ...",chỗ để xe
2,: 24 tỷ,128 m\n2,"7,5m",3 lầu,3 phòng ngủ,"Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận...",chỗ để xe
3,": 25,5 tỷ",165 m\n2,"7,5m",1 lầu,1 phòng ngủ,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",0
4,: 24 tỷ,320 m\n2,30m,5 lầu,14 phòng ngủ,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
...,...,...,...,...,...,...,...
7796,": 4,8 tỷ",60 m\n2,"5,5m",4 lầu,2 phòng ngủ,"Đường Điện Biên Phủ, Phường Chính Gián, Quận T...",chỗ để xe
7797,": 5,8 tỷ",65 m\n2,5 5m,4 lầu,3 phòng ngủ,"Đường Mai Lão Bạng, Phường Thuận Phước, Quận...",0
7798,": 5,7 tỷ",65 m\n2,11m,2 lầu,2 phòng ngủ,"Đường Trương Định, Phường Mân Thái, Quận Sơn Trà,",0
7799,": 9,3 tỷ",100 m\n2,6m,2 lầu,3 phòng ngủ,"Đường Trần Khánh Dư, Phường Mỹ An, Quận Ngũ Hà...",0


In [20]:
def clean_price(price, area):
    if isinstance(price, str):
        price = price.replace(':', '').replace(',', '.').strip()

        if 'ngàn' in price:
            return None
        
        if 'triệu / m2' in price and isinstance(area, str):
            try:
                area_value = clean_area(area)
                price_value = float(price.replace('triệu/m2', '').strip())
                return price_value * area_value / 1000 
            except:
                return None

        if 'tỷ' in price:
            try:
                return float(price.replace('tỷ', '').strip())
            except:
                return None
        
        if '/ m2' in price:
            try:
                price_value = price.split('/')[0].strip()
                return float(price_value)
            except:
                return None

    return None
def clean_area(area):
    if isinstance(area, str):
        area = area.replace('\n', '').replace('m', '').replace('2', '').strip() 
        try:
            return float(area)
        except:
            return None
    return None

def clean_numeric_column(value):
    if isinstance(value, str):
        try:
            return float(''.join([c for c in value if c.isdigit() or c == '.']))
        except:
            return None
    return None

def clean_garage(garage):
    if isinstance(garage, str) and "0" not in garage:
        return 1
    return 0

In [21]:
df['price'] = df.apply(lambda row: clean_price(row['price'], row['area']), axis=1)
df['area'] = df['area'].apply(clean_area)
df['garage'] = df['garage'].apply(clean_garage)
df['road_width'] = df['road_width'].apply(clean_numeric_column)
df['bedroom'] = df['bedroom'].apply(clean_numeric_column)
df['floors'] = df['floors'].apply(clean_numeric_column)


In [22]:
df['bedroom'].unique()

array([  2.,   3.,   1.,  14.,   4.,  10.,  nan,  11.,   5.,   6.,  12.,
         8.,   9.,  15.,  27.,  16.,   7.,  20.,  25.,  13.,  28.,  21.,
        19.,  17.,  23.,  18.,  33.,  26.,  30.,  22.,  34.,  36., 150.,
        51.,  24.])

In [23]:
# df['price']=df['price'].fillna(df['price'].mean())
# df['area']=df['area'].fillna(df['area'].mean())
df['road_width']=df['road_width'].fillna(df['road_width'].mean())
df['floors']=df['floors'].fillna(int(df['floors'].mean()))
df['bedroom']=df['bedroom'].fillna(int(df['bedroom'].mean()))

In [24]:
df=df.dropna()

In [25]:
df

,price,area,road_width,floors,bedroom,loc,garage
0,3.8,50.0,7.0,2.0,2.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
1,9.7,100.0,6.0,2.0,3.0,"Đường Trần Tống, Phường Thạc Gián, Quận Thanh ...",1
2,24.0,18.0,75.0,3.0,3.0,"Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận...",1
3,25.5,165.0,75.0,1.0,1.0,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",0
4,24.0,30.0,30.0,5.0,14.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0
...,...,...,...,...,...,...,...
7796,4.8,60.0,55.0,4.0,2.0,"Đường Điện Biên Phủ, Phường Chính Gián, Quận T...",1
7797,5.8,65.0,55.0,4.0,3.0,"Đường Mai Lão Bạng, Phường Thuận Phước, Quận...",0
7798,5.7,65.0,11.0,2.0,2.0,"Đường Trương Định, Phường Mân Thái, Quận Sơn Trà,",0
7799,9.3,100.0,6.0,2.0,3.0,"Đường Trần Khánh Dư, Phường Mỹ An, Quận Ngũ Hà...",0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7772 entries, 0 to 7800
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   price       7772 non-null   float64
 1   area        7772 non-null   float64
 2   road_width  7772 non-null   float64
 3   floors      7772 non-null   float64
 4   bedroom     7772 non-null   float64
 5   loc         7772 non-null   object 
 6   garage      7772 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 485.8+ KB


In [67]:
df.to_csv('cleaned_df_alonhadat.csv',encoding='utf-8',index=False)

In [12]:
# def remove_outlier_usingIQR(df,column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [15]:
# columns_to_clean = ['area','price']
# for column in columns_to_clean:
#     cleaned_df = remove_outlier_usingIQR(cleaned_df, column)
# cleaned_df.describe()

,price,area,road_width,floors,bedroom,garage
count,7668.000000,7668.000000,7668.000000,7668.000000,7668.000000,7668.000000
mean,5.980797,72.287024,28.056729,2.539515,3.639150,0.629630
std,3.180411,33.786054,30.549731,1.072622,5.240462,0.482935
min,1.379000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,3.500000,55.000000,5.000000,2.000000,3.000000,0.000000
50%,5.200000,75.000000,10.000000,2.000000,3.000000,1.000000
75%,7.500000,99.000000,55.000000,3.000000,4.000000,1.000000
max,15.800000,167.000000,545.000000,11.000000,410.000000,1.000000


## Getting latitude and longitude from geopy - nominatim

In [51]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


geolocator = Nominatim(user_agent="lengocduy169@gmail.com", timeout = 10)
geoadd = RateLimiter(geolocator.geocode, min_delay_seconds= 1)

df['location'] = df.loc[:, 'loc'].apply(geoadd) 

df[:20]


/tmp/ipykernel_70967/746322651.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location'] = df.loc[:, 'loc'].apply(geoadd)  # Correct indexing, as slicing is zero-indexed


,price,area,road_width,floors,bedroom,loc,garage,location
0,3.80,50.0,7.0,2.0,2.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0,"(Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Qu..."
1,9.70,100.0,6.0,2.0,3.0,"Đường Trần Tống, Phường Thạc Gián, Quận Thanh ...",1,"(Đường Trần Tống, Phường Thạc Gián, Thanh Khê,..."
2,24.00,18.0,75.0,3.0,3.0,"Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận...",1,"(Đường Dương Đình Nghệ, Phường An Hải Bắc, Quậ..."
3,25.50,165.0,75.0,1.0,1.0,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",0,"(Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hà..."
4,24.00,30.0,30.0,5.0,14.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0,"(Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Qu..."
5,28.00,165.0,75.0,1.0,1.0,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",1,"(Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hà..."
6,6.05,100.0,75.0,2.0,3.0,"Đường Giang Châu 1, Phường Khuê Mỹ, Quận Ngũ H...",0,"(Đường Giang Châu 1, Phường Khuê Mỹ, Quận Ngũ ..."
7,24.00,147.0,15.0,3.0,3.0,"Đường Đống Đa, Phường Thuận Phước, Quận Hải Châu,",1,"(Đường Đống Đa, Phường Thuận Phước, Quận Hải C..."
8,5.60,90.0,75.0,3.0,4.0,"Đường Lê Đại Hành, Phường Khuê Trung, Quận Cẩm...",0,"(Đường Lê Đại Hành, Phường Khuê Trung, Quận Cẩ..."
9,20.50,170.0,15.0,1.0,1.0,"Đường Hồ Nghinh, Phường Phước Mỹ, Quận Sơn Trà,",1,"(Đường Hồ Nghinh, Phường Phước Mỹ, Quận Sơn Tr..."


In [52]:
df.loc[:,'latitude'] = df.loc[:,'location'].apply(lambda loc: loc.latitude if loc else None)
df.loc[:,'longitude'] = df.loc[:,'location'].apply(lambda loc: loc.longitude if loc else None)

/tmp/ipykernel_70967/438653058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'latitude'] = df.loc[:,'location'].apply(lambda loc: loc.latitude if loc else None)
/tmp/ipykernel_70967/438653058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'longitude'] = df.loc[:,'location'].apply(lambda loc: loc.longitude if loc else None)


In [53]:
df

,price,area,road_width,floors,bedroom,loc,garage,location,latitude,longitude
0,3.800,50.0,7.0,2.0,2.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0,"(Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Qu...",16.083811,108.228874
1,9.700,100.0,6.0,2.0,3.0,"Đường Trần Tống, Phường Thạc Gián, Quận Thanh ...",1,"(Đường Trần Tống, Phường Thạc Gián, Thanh Khê,...",16.058643,108.210940
2,24.000,18.0,75.0,3.0,3.0,"Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận...",1,"(Đường Dương Đình Nghệ, Phường An Hải Bắc, Quậ...",16.068177,108.239219
3,25.500,165.0,75.0,1.0,1.0,"Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hàn...",0,"(Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hà...",16.052361,108.244859
4,24.000,30.0,30.0,5.0,14.0,"Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quậ...",0,"(Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Qu...",16.083811,108.228874
...,...,...,...,...,...,...,...,...,...,...
8628,4.600,4.0,55.0,4.0,4.0,"Đường Hoàng Kế Viêm, Phường Mỹ An, Quận Ngũ Hà...",1,"(Đường Hoàng Kế Viêm, Phường Mỹ An, Quận Ngũ H...",16.049818,108.247122
8629,8.499,70.0,55.0,2.0,3.0,"Đường An Thượng 17, Phường Mỹ An, Quận Ngũ Hàn...",1,"(Đường An Thượng 17, Phường Mỹ An, Quận Ngũ Hà...",16.049983,108.241119
8630,10.000,100.0,5.0,5.0,10.0,"Đường Chế Lan Viên, Phường Mỹ An, Quận Ngũ Hàn...",1,"(Đường Chế Lan Viên, Phường Mỹ An, Quận Ngũ Hà...",16.043737,108.244963
8631,9.300,90.0,18.0,3.0,4.0,"Đường Hồ Xuân Hương, Phường Khuê Mỹ, Quận Ngũ ...",1,"(Đường Hồ Xuân Hương, Phường Khuê Mỹ, Quận Ngũ...",16.036454,108.240234


In [ ]:
# df = df.drop(['location'])

## IMPORTING DATA INTO DATABASE

In [34]:
import pymysql


timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="Housing_Real_Estate",
  host="mysql-for-ady201m-project-ady201m-nhom5.e.aivencloud.com",
  password="AVNS_Z2RBipwQfoNZD7N0O11",
  read_timeout=timeout,
  port=22064,
  user="avnadmin",
  write_timeout=timeout,
)


In [35]:
cursor = connection.cursor()


In [29]:
cursor.execute("CREATE DATABASE IF NOT EXISTS Housing_Real_Estate")
    
cursor.execute("USE Housing_Real_Estate")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Housing_Data_1 (
        id INT PRIMARY KEY AUTO_INCREMENT,
        price FLOAT,
        area FLOAT,
        road_width FLOAT,
        floors FLOAT,
        bedroom INT,
        garage INT,
        loc TEXT
    )
""")

for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Housing_Data_1
        (price, area, road_width, floors, bedroom, garage, loc)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['price'], 
        row['area'], 
        row['road_width'], 
        row['floors'], 
        row['bedroom'], 
        row['garage'], 
        row['loc']
    ))

In [30]:
connection.commit()

In [36]:
cursor.execute("USE Housing_Real_Estate")
cursor.execute("SELECT * FROM Housing_Data_1")
results = cursor.fetchall()
for result in results:
    print(result)

{'id': 1636, 'price': 3.8, 'area': 50.0, 'road_width': 7.0, 'floors': 2.0, 'bedroom': 2, 'garage': 0, 'loc': 'Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quận Sơn Trà,'}
{'id': 1637, 'price': 9.7, 'area': 100.0, 'road_width': 6.0, 'floors': 2.0, 'bedroom': 3, 'garage': 1, 'loc': 'Đường Trần Tống, Phường Thạc Gián, Quận Thanh Khê,'}
{'id': 1638, 'price': 24.0, 'area': 18.0, 'road_width': 75.0, 'floors': 3.0, 'bedroom': 3, 'garage': 1, 'loc': 'Đường Dương Đình Nghệ, Phường An Hải Bắc, Quận Sơn Trà,'}
{'id': 1639, 'price': 25.5, 'area': 165.0, 'road_width': 75.0, 'floors': 1.0, 'bedroom': 1, 'garage': 0, 'loc': 'Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ Hành Sơn,'}
{'id': 1640, 'price': 24.0, 'area': 30.0, 'road_width': 30.0, 'floors': 5.0, 'bedroom': 14, 'garage': 0, 'loc': 'Đường Trần Hưng Đạo, Phường Nại Hiên Đông, Quận Sơn Trà,'}
{'id': 1641, 'price': 28.0, 'area': 165.0, 'road_width': 75.0, 'floors': 1.0, 'bedroom': 1, 'garage': 1, 'loc': 'Đường Lê Quang Đạo, Phường Mỹ An, Quận Ngũ H

In [ ]:
connection.close()